In [2]:
import cvxpy as cp
import numpy as np
from collections import defaultdict

# Parameters
E = 20  # Number of guests
S = 100  # Cost per single room
D = 135  # Cost per double room
R = 0.1  # 10% senior discount
B = 5000  # Budget
N = 1  # Number of nights

# Controlled Data Case
single_pref_controlled = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])  # 1 if prefers single room
genders_controlled = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])  # 1 for female, 0 for male
ages_controlled = np.array([30, 65, 74, 61, 29, 55, 60, 67, 34, 80, 45, 70, 59, 62, 58, 63, 64, 55, 66, 68])  # Ages

# Random Data Case
np.random.seed(42)  # For reproducibility
single_pref_random = np.random.randint(0, 2, size=E)
genders_random = np.random.randint(0, 2, size=E)
ages_random = np.random.randint(20, 80, size=E)

# Choose which data to use: controlled or random
single_pref = single_pref_controlled  # or single_pref_random
genders = genders_controlled  # or genders_random
ages = ages_controlled  # or ages_random

# Create pairs for double rooms
pairs = [(i, j) for i in range(E) for j in range(i + 1, E)]
num_pairs = len(pairs)

# Mapping from (i, j) to index in y
pair_indices = {pair: idx for idx, pair in enumerate(pairs)}

# Variables
x = cp.Variable(E, boolean=True)  # Single room assignments
y = cp.Variable(num_pairs, boolean=True)  # Double room pairings
z = cp.Variable(E, boolean=True)  # Senior discount eligibility

# Build person-to-y indices mapping
person_to_y_indices = defaultdict(list)
for idx, (i, j) in enumerate(pairs):
    person_to_y_indices[i].append(idx)
    person_to_y_indices[j].append(idx)

# Constraints
constraints = []

# Each person must be assigned to exactly one room
for i in range(E):
    constraints.append(x[i] + cp.sum([y[idx] for idx in person_to_y_indices[i]]) == 1)

# Respect single room preferences
for i in range(E):
    constraints.append(x[i] >= single_pref[i])

# Seniors must be eligible for discounts only if in a double room
for i in range(E):
    if ages[i] >= 60:
        constraints.append(z[i] <= cp.sum([y[idx] for idx in person_to_y_indices[i]]))
    else:
        constraints.append(z[i] == 0)

# Ensure gender compatibility for double rooms
for idx, (i, j) in enumerate(pairs):
    if genders[i] != genders[j]:
        constraints.append(y[idx] == 0)

# Force at least one senior to a double room for the discount if possible
num_seniors_no_single_pref = sum(1 for i in range(E) if ages[i] >= 60 and single_pref[i] == 0)
if num_seniors_no_single_pref >= 1:
    constraints.append(cp.sum([z[i] for i in range(E) if ages[i] >= 60]) >= 1)

# Budget constraint
total_cost = (
    N * S * cp.sum(x) +
    N * D * cp.sum(y) -
    R * D * cp.sum(z)
)
constraints.append(total_cost <= B)

# Objective Function
objective = cp.Minimize(total_cost)

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)

# Output Results
print("\n--- Solution Summary ---")
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    print(f"Minimum Total Cost: {problem.value}")
    
    print("\nSingle Room Assignments:")
    single_room_assigned = []
    for i in range(E):
        if x[i].value > 0.5:
            print(f"Person {i + 1}: Single Room")
            single_room_assigned.append(i)

    print("\nDouble Room Pairings:")
    double_room_count = 0
    double_room_pairings = []
    for idx, (i, j) in enumerate(pairs):
        if y[idx].value > 0.5:
            print(f"Persons {i + 1} and {j + 1}: Double Room")
            double_room_count += 1
            double_room_pairings.append((i, j))

    print("\nSeniors (age 60+) Assigned to Double Rooms for Discount:")
    senior_discount_count = 0
    for i in range(E):
        if ages[i] >= 60:
            if z[i].value > 0.5:
                print(f"Senior Person {i + 1} (age {ages[i]}) assigned to double room for discount.")
                senior_discount_count += 1
            else:
                print(f"Senior Person {i + 1} (age {ages[i]}) not assigned to a double room.")

    print("\n--- Budget Analysis ---")
    single_room_cost = N * S * sum(x[i].value for i in range(E))
    double_room_cost = N * D * sum(y[idx].value for idx in range(num_pairs))
    senior_discount = R * D * sum(z[i].value for i in range(E))
    final_cost = single_room_cost + double_room_cost - senior_discount
    print(f"Single Rooms Total Cost: ${single_room_cost}")
    print(f"Double Rooms Total Cost: ${double_room_cost}")
    print(f"Senior Discount Total: -${senior_discount}")
    print(f"Final Total Cost: ${final_cost}")
    print(f"Budget Allowed: ${B}")
    print(f"Within Budget: {'Yes' if final_cost <= B else 'No'}")

    print("\n--- Summary ---")
    print(f"Total Single Rooms Assigned: {len(single_room_assigned)}")
    print(f"Total Double Rooms Assigned: {double_room_count}")
    print(f"Seniors Assigned to Double Rooms (for discount): {senior_discount_count}")
else:
    print("No optimal solution found.")



--- Solution Summary ---
Minimum Total Cost: 1594.0

Single Room Assignments:
Person 1: Single Room
Person 3: Single Room
Person 5: Single Room
Person 7: Single Room
Person 9: Single Room
Person 12: Single Room
Person 14: Single Room
Person 16: Single Room
Person 18: Single Room
Person 20: Single Room

Double Room Pairings:
Persons 2 and 19: Double Room
Persons 4 and 6: Double Room
Persons 8 and 13: Double Room
Persons 10 and 15: Double Room
Persons 11 and 17: Double Room

Seniors (age 60+) Assigned to Double Rooms for Discount:
Senior Person 2 (age 65) assigned to double room for discount.
Senior Person 3 (age 74) not assigned to a double room.
Senior Person 4 (age 61) assigned to double room for discount.
Senior Person 7 (age 60) not assigned to a double room.
Senior Person 8 (age 67) assigned to double room for discount.
Senior Person 10 (age 80) assigned to double room for discount.
Senior Person 12 (age 70) not assigned to a double room.
Senior Person 14 (age 62) not assigned to 

In [2]:
import cvxpy as cp
import numpy as np
from collections import defaultdict

# Parameters
E = 40  # Number of people
S = 100  # Cost per single room
D = 135  # Cost per double room
R = 0.1  # 10% senior discount
B = 8000  # Adjusted budget for larger group
N = 1  # Number of nights

# Random Data Generation
np.random.seed(42)  # For reproducibility
single_pref = np.random.randint(0, 2, size=E)  # 1 if prefers single room
genders = np.random.randint(0, 2, size=E)  # 1 for female, 0 for male
ages = np.random.randint(20, 80, size=E)  # Ages between 20 and 79

# Create pairs for double rooms
pairs = [(i, j) for i in range(E) for j in range(i + 1, E)
         if genders[i] == genders[j]]  # Only same-gender pairs
num_pairs = len(pairs)

# Variables
x = cp.Variable(E, boolean=True)  # Single room assignments
y = cp.Variable(num_pairs, boolean=True)  # Double room pairings
z = cp.Variable(E, boolean=True)  # Senior discount eligibility

# Build person-to-y indices mapping
person_to_y_indices = defaultdict(list)
for idx, (i, j) in enumerate(pairs):
    person_to_y_indices[i].append(idx)
    person_to_y_indices[j].append(idx)

# Constraints
constraints = []

# Each person must be assigned to exactly one room
for i in range(E):
    constraints.append(
        x[i] + cp.sum([y[idx] for idx in person_to_y_indices[i]]) == 1
    )

# Respect single room preferences
for i in range(E):
    constraints.append(x[i] >= single_pref[i])

# Seniors must be eligible for discounts only if in a double room
for i in range(E):
    if ages[i] >= 60:
        constraints.append(
            z[i] <= cp.sum([y[idx] for idx in person_to_y_indices[i]])
        )
    else:
        constraints.append(z[i] == 0)

# Force at least one senior to a double room for the discount if possible
num_seniors_no_single_pref = sum(
    1 for i in range(E) if ages[i] >= 60 and single_pref[i] == 0
)
if num_seniors_no_single_pref >= 1:
    constraints.append(cp.sum([z[i] for i in range(E) if ages[i] >= 60]) >= 1)

# Budget constraint
total_cost = (
    N * S * cp.sum(x)
    + N * D * cp.sum(y)
    - R * D * cp.sum(z)
)
constraints.append(total_cost <= B)

# Objective Function
objective = cp.Minimize(total_cost)

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)

# Output Results
print("\n--- Solution Summary ---")
if problem.status == cp.OPTIMAL:
    print(f"Minimum Total Cost: {problem.value}")

    print("\nSingle Room Assignments:")
    single_room_assigned = []
    for i in range(E):
        if x[i].value > 0.5:
            print(f"Person {i+1}: Single Room")
            single_room_assigned.append(i)

    print("\nDouble Room Pairings:")
    double_room_count = 0
    for idx, (i, j) in enumerate(pairs):
        if y[idx].value > 0.5:
            print(f"Persons {i+1} and {j+1}: Double Room")
            double_room_count += 1

    print("\nSeniors (age 60+) Assigned to Double Rooms for Discount:")
    senior_discount_count = 0
    for i in range(E):
        if ages[i] >= 60:
            if z[i].value > 0.5:
                print(f"Senior Person {i+1} (age {ages[i]}) assigned to double room for discount.")
                senior_discount_count += 1
            else:
                print(f"Senior Person {i+1} (age {ages[i]}) not assigned to a double room.")

    print("\n--- Budget Analysis ---")
    single_room_cost = N * S * sum(x[i].value for i in range(E))
    double_room_cost = N * D * sum(y[idx].value for idx in range(num_pairs))
    senior_discount = R * D * sum(z[i].value for i in range(E))
    final_cost = single_room_cost + double_room_cost - senior_discount

    print(f"Single Rooms Total Cost: ${single_room_cost}")
    print(f"Double Rooms Total Cost: ${double_room_cost}")
    print(f"Senior Discount Total: -${senior_discount}")
    print(f"Final Total Cost: ${final_cost}")
    print(f"Budget Allowed: ${B}")
    print(f"Within Budget: {'Yes' if final_cost <= B else 'No'}")

    print("\n--- Summary ---")
    print(f"Total Single Rooms Assigned: {len(single_room_assigned)}")
    print(f"Total Double Rooms Assigned: {double_room_count}")
    print(f"Seniors Assigned to Double Rooms (for discount): {senior_discount_count}")
else:
    print("No optimal solution found.")



--- Solution Summary ---
Minimum Total Cost: 3334.0

Single Room Assignments:
Person 2: Single Room
Person 6: Single Room
Person 7: Single Room
Person 10: Single Room
Person 15: Single Room
Person 17: Single Room
Person 18: Single Room
Person 19: Single Room
Person 21: Single Room
Person 22: Single Room
Person 23: Single Room
Person 24: Single Room
Person 25: Single Room
Person 26: Single Room
Person 27: Single Room
Person 28: Single Room
Person 29: Single Room
Person 30: Single Room
Person 33: Single Room
Person 34: Single Room
Person 35: Single Room
Person 37: Single Room

Double Room Pairings:
Persons 1 and 8: Double Room
Persons 3 and 9: Double Room
Persons 4 and 5: Double Room
Persons 11 and 20: Double Room
Persons 12 and 14: Double Room
Persons 13 and 39: Double Room
Persons 16 and 31: Double Room
Persons 32 and 36: Double Room
Persons 38 and 40: Double Room

Seniors (age 60+) Assigned to Double Rooms for Discount:
Senior Person 2 (age 63) not assigned to a double room.
Senior P